### Aprendizado Ensemble - Florestas Aleatórias

In [1]:
# bibliotecas utilizadas no desenvolvimento do projeto
from collections import Counter, defaultdict, OrderedDict
from functools import partial
import math, random
import pandas as pd
from csv import reader
import numpy as np
from random import seed
from random import randrange
from graphviz import Digraph

In [2]:
def build_dataset(path_file): 
    dataset = list()
    with open(path_file, 'r') as file:
        csv_reader = reader(file)
        rows = [r for r in csv_reader]
    # capturando os cabeçalhos
    headers = rows[0]
    
    remove_row_with_values_missing(rows, headers)
    
    for row in rows[1:len(rows)]:
        out_dic = {}
        for header_index in range(len(headers)):
            out_dic.update({ headers[header_index]: row[header_index]})
        
        # captura a classe
        class_key = list(out_dic.items())[-1]
        # remove a classe captura do dicionario
        del out_dic[list(out_dic.keys())[-1]]
        # montando o dataset de saída
        data_tuple = (out_dic, class_key[1])
        dataset.append(data_tuple)
    return dataset

# TODO: devemos remover linhas com dados faltando?
def remove_row_with_values_missing(rows, headers):
    quant = 0
    for row in rows[1:len(rows)]:
        for header_index in range(len(headers)):
            if(row[header_index] is None or row[header_index] == ""):
                quant = quant + 1
    

# construindo a base de dados a partir do arquivo fornecido
dataset = build_dataset('datasets/benchmark.csv')

1. Desenvolvimento do algoritmo de indução de uma árvore de decisão, usando como critério de seleção de atributos para divisão de nós o **Ganho de Informação (baseado no conceito de entropia)**. **Tratando tanto atributos categóricos quanto numéricos.**

In [3]:

def entropy(class_probabilities):
    """given a list of class probabilities, compute the entropy"""
    return sum(-p * math.log(p, 2) for p in class_probabilities if p)

def class_probabilities(labels):
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]

def data_entropy(labeled_data):
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

def partition_entropy(subsets):
    """find the entropy from this partition of data into subsets"""
    total_count = sum(len(subset) for subset in subsets)

    return sum( data_entropy(subset) * len(subset) / total_count for subset in subsets )

def group_by(items, key_fn):
    """returns a defaultdict(list), where each input item is in the list whose key is key_fn(item)"""
    groups = defaultdict(list)
    for item in items:
        key = key_fn(item)
        groups[key].append(item)
    return groups

def partition_by(inputs, attribute):
    """returns a dict of inputs partitioned by the attribute each input is a pair (attribute_dict, label)"""
    return group_by(inputs, lambda x: x[0][attribute])

def partition_entropy_by(inputs, attribute):
    """computes the entropy corresponding to the given partition"""
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values()) # retorna a media dos sub-conjuntos resultantes

def gain(class_entropy, mean_entropy):
    return class_entropy - mean_entropy;

def class_entropy(inputs):
    """retorna a entropia sobre a classe"""
    class_values = []
    for item in inputs:
        class_values.append(item[1]) # suponto a definição atual dos dados!
    probabilities = class_probabilities(class_values)
    class_entropy = entropy(probabilities)
    return class_entropy

def diff_class(inputs):
    classes = []
    for current_class in list(set([label for item, label in inputs if label])):
        classes.append({'class': current_class, 'amount': 0})
    for current_class in [label for item, label in inputs if label]:
        for my_class in classes:
            if(my_class['class']  == current_class):
                my_class['amount'] = my_class['amount'] + 1
    return classes

def highest_class(classes):
    my_class = classes[0]
    for current_class in classes:
        if(current_class['amount'] > my_class['amount']):
            my_class = current_class
    return my_class
    

def build_tree(inputs, split_candidates=None):

    # if this is our first pass,
    # all keys of the first input are split candidates
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()
              
    # obtendo as classes
    # [(classe, quantidade)]
    classes = diff_class(inputs)
    
    # count Trues and Falses in the inputs
    num_inputs = len(inputs)
    num_trues = len([label for item, label in inputs if label])
    num_falses = num_inputs - num_trues
    
    # verificando a homogeneidade do cojunto
    if(len(classes) == 1):
        return classes[0]['class']

    if not split_candidates:          # if no split candidates left
        return highest_class(classes) # return the majority leaf

    # otherwise, split on the best attribute
    
    best_gain = 0
    best_attribute = None
    for candidate in split_candidates:
        mean_entropy = partition_entropy_by(inputs, candidate)
        print('[INFO] Atributo: ', candidate, '- entropia média: ', mean_entropy)
        gain_result = gain(class_entropy(inputs), mean_entropy)
        print('[INFO] Atributo: ', candidate, '- ganho: ', gain_result)
        if(gain_result > best_gain):
            best_gain = gain(class_entropy(inputs), mean_entropy)
            best_attribute = candidate
        print('--')
    
    print('----------------------------------------')
    print('[INFO] best_attribute: ', best_attribute)
    print('----------------------------------------')
    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates
                      if a != best_attribute]

    # recursively build the subtrees
    subtrees = { attribute : build_tree(subset, new_candidates)
                 for attribute, subset in partitions.items() }

    #subtrees[None] = num_trues > num_falses # default case

    return (best_attribute, subtrees)

if __name__ == "__main__":
    # construindo a árvore sobre os dados construindos anteriormente
    tree = build_tree(dataset)
    print('\n')
    print('----------------------------------------')
    print("[INFO] Árvore construida:")
    print('----------------------------------------')
    print(tree)

[INFO] Atributo:  Tempo - entropia média:  0.6935361388961919
[INFO] Atributo:  Tempo - ganho:  0.246749819774439
--
[INFO] Atributo:  Temperatura - entropia média:  0.9110633930116763
[INFO] Atributo:  Temperatura - ganho:  0.029222565658954647
--
[INFO] Atributo:  Umidade - entropia média:  0.7884504573082896
[INFO] Atributo:  Umidade - ganho:  0.15183550136234136
--
[INFO] Atributo:  Ventoso - entropia média:  0.8921589282623617
[INFO] Atributo:  Ventoso - ganho:  0.04812703040826927
--
----------------------------------------
[INFO] best_attribute:  Tempo
----------------------------------------
[INFO] Atributo:  Temperatura - entropia média:  0.4
[INFO] Atributo:  Temperatura - ganho:  0.5709505944546686
--
[INFO] Atributo:  Umidade - entropia média:  0.0
[INFO] Atributo:  Umidade - ganho:  0.9709505944546686
--
[INFO] Atributo:  Ventoso - entropia média:  0.9509775004326938
[INFO] Atributo:  Ventoso - ganho:  0.01997309402197478
--
----------------------------------------
[INFO] 

In [4]:
g = Digraph('G', filename='tree.gv')

g.edge('Tempo, ganho 0.970', 'Umidade', label='Ensolarado, ganho: 0.970')
g.edge('Umidade', 'Não', label='Alta')
g.edge('Umidade', 'Sim', label='Normal')
g.edge('Tempo, ganho 0.970', 'Sim', label='Nublado')
g.edge('Tempo, ganho 0.970', 'Ventoso', label='Nublado, ganho: 0.970')
g.edge('Ventoso', 'Sim', label='Falso')
g.edge('Ventoso', 'Nao', label='Verdadeiro')

#g.view()

2. Uma função para percorrer a árvore de decisão treinada e realizar a classificação de uma nova instância (do conjunto de teste);

In [5]:
def classify(tree, classes, input):
    """classificar a entrada usando a árvore de decisão fornecida"""

    # se este é um nó folha, retorna seu valor
    if tree in classes:
        return tree

    # caso contrário, encontre a subárvore correta
    attribute, subtree_dict = tree

    subtree_key = input.get(attribute)  # Nenhum se a entrada estiver faltando atributo

    if subtree_key not in subtree_dict: # se não houver subárvore para a chave
        subtree_key = None              # vamos usar a subárvore não

    subtree = subtree_dict[subtree_key] # escolha a subárvore apropriada
    return classify(subtree, classes, input) # e usá-lo para classificar a entrada


# obtendo as classes possiveis para a àrvore
classes = list(set([label for item, label in dataset if label]))

print('\n')
print('[INFO] Dados de amostras (ultimas duas linhas) para teste de resultado:')

print('Nublado / Quente / Normal / Falso ->', classify(tree, classes,
    { 'Tempo' : 'Nublado',
      'Temperatura' : 'Quente',
      'Umidade' : 'Normal',
      'Ventoso' : 'Falso'} ))
print('Chuvoso / Amena / Alta / Verdadeiro ->', classify(tree, classes,
    { 'Tempo' : 'Chuvoso',
      'Temperatura' : 'Amena',
      'Umidade' : 'Alta',
      'Ventoso' : 'Verdadeiro'} ))



[INFO] Dados de amostras (ultimas duas linhas) para teste de resultado:
Nublado / Quente / Normal / Falso -> Sim
Chuvoso / Amena / Alta / Verdadeiro -> Nao


3. O mecanismo de bootstrap (amostragem com reposição) para geração de subconjuntos a partir do conjunto de dados de treinamento originais. Cada bootstrap será utilizado para o treinamento de uma árvore no aprendizado ensemble

In [6]:
def bootstrap(dataset):
    bootstrap = dataset.sample(n = len(dataset), replace = True)
    return bootstrap

result_bootstrap = bootstrap(pd.read_csv('datasets/voting.csv'))

In [7]:
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

result_cross = cross_validation_split(result_bootstrap.values.tolist(), 10)

In [8]:
# TODO: DIVISÃO TREINO E TESTE
def train_split(dataset, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        return train_set

def test_split(dataset, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    for fold in folds:
        test_set = list(fold)
        return test_set


# Não está retendo os dados do banco de bootstrap, é os mesmos dados, mas ainda tá flutuando e gerando listas diferentes, temos que discutir se isso tem algum problema 


test_set = test_split(result_cross, 10) 
print('[INFO] definindo folds para teste')

print('\n[INFO] definindo folds para treino')
train_set = train_split(result_cross, 10)

[INFO] definindo folds para teste

[INFO] definindo folds para treino


In [9]:
def build_from_split(headers, dataset_from_bootstrap): 
    new_dataset = list()    
    for row in dataset_from_bootstrap[0:len(dataset_from_bootstrap)]:
        out_dic = {}
        for header_index in range(len(headers)):
            out_dic.update({ headers[header_index]: row[header_index]})
        
        # captura a classe
        class_key = list(out_dic.items())[-1]
        # remove a classe captura do dicionario
        del out_dic[list(out_dic.keys())[-1]]
        # montando o dataset de saída
        data_tuple = (out_dic, class_key[1])
        new_dataset.append(data_tuple)
    return new_dataset

# obtebdo os cabeçalhos
headers = result_bootstrap.columns.tolist()
#print(build_tree(build_from_bootstrap(headers, result_bootstrap.values.tolist())))


In [10]:
def build_pre_dataset(old_dataset):
    new_dataset = list()
    for i in old_dataset:
        for j in i:
            new_dataset.append(j)
    return new_dataset

In [11]:
# construindo a árvore a partir do split vindo do cross
new_train = build_pre_dataset(train_set) # treino

new_tree_from_train = build_tree(build_from_split(headers, new_train))

# construindo a árvore a partir do split vindo do cross
new_test = build_pre_dataset(test_set) # teste
        
classes = list(set([label for item, label in build_from_split(headers, new_train) if label]))
     
classe_teste = list()
preditos_teste = list()
for current in build_from_split(headers, new_test):
    preditos_teste.append(current[1])
    classe_teste.append(classify(new_tree_from_train, classes, current[0]))
  
#print('\n[INFO] resultado real dos dados de teste: ')
#print(classe_teste)
#print('\n[INFO] resultado preditos esperado: ')
#print(preditos_teste)

[INFO] Atributo:  handicapped-infants - entropia média:  0.8498982165269577
[INFO] Atributo:  handicapped-infants - ganho:  0.12837163833317433
--
[INFO] Atributo:  water-project-cost-sharing - entropia média:  0.9743548925747074
[INFO] Atributo:  water-project-cost-sharing - ganho:  0.00391496228542465
--
[INFO] Atributo:  adoption-of-the-budget-resolution - entropia média:  0.5741763181345978
[INFO] Atributo:  adoption-of-the-budget-resolution - ganho:  0.4040935367255343
--
[INFO] Atributo:  physician-fee-freeze - entropia média:  0.235918490457937
[INFO] Atributo:  physician-fee-freeze - ganho:  0.7423513644021951
--
[INFO] Atributo:  el-salvador-aid - entropia média:  0.5772960803759168
[INFO] Atributo:  el-salvador-aid - ganho:  0.40097377448421523
--
[INFO] Atributo:  religious-groups-in-schools - entropia média:  0.865645075995777
[INFO] Atributo:  religious-groups-in-schools - ganho:  0.11262477886435507
--
[INFO] Atributo:  anti-satellite-test-ban - entropia média:  0.8131542

[INFO] Atributo:  synfuels-corporation-cutback - entropia média:  0.03950796688566967
[INFO] Atributo:  synfuels-corporation-cutback - ganho:  0.005341647265700952
--
[INFO] Atributo:  education-spending - entropia média:  0.04379200144324535
[INFO] Atributo:  education-spending - ganho:  0.0010576127081252731
--
[INFO] Atributo:  superfund-right-to-sue - entropia média:  0.04214992154275959
[INFO] Atributo:  superfund-right-to-sue - ganho:  0.002699692608611033
--
[INFO] Atributo:  crime - entropia média:  0.03602483839876575
[INFO] Atributo:  crime - ganho:  0.008824775752604867
--
[INFO] Atributo:  duty-free-exports - entropia média:  0.03724937767711321
[INFO] Atributo:  duty-free-exports - ganho:  0.0076002364742574074
--
[INFO] Atributo:  export-administration-act-south-africa - entropia média:  0.04183517323321366
[INFO] Atributo:  export-administration-act-south-africa - ganho:  0.0030144409181569604
--
----------------------------------------
[INFO] best_attribute:  adoption-o

In [12]:
matrix = list()
accuracy = list()
recall = list()
precision = list()
score = list()

def classification_report(classe_teste, preditos_teste):
    true_class = 'democrat'
    negative_class = 'republican'
    # declarando a frêquencia
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    
    for (indice, v_real) in enumerate(classe_teste):
        v_predito = preditos_teste[indice]
        # se trata de um valor real da classe positiva
        if v_real == true_class:
            tp += 1 if v_predito == v_real else 0
            fp += 1 if v_predito != v_real else 0
        else:
            tn += 1 if v_predito == v_real else 0
            fn += 1 if v_predito != v_real else 0
                
    # calculando os valores de frêquencia
    accuracy.append((tp+tn)/(tp+fp+tn+fn))
    recall.append(tp/(tp+fn))
    precision.append(tp/(tp+fp))
    score.append(2*tp/(2*tp+fp+fn))
    
    # desenhando a matriz de confusão
    print(np.array([
        [ tp, fp ],
        [ fn, tn ]
    ]))
    
classification_report(classe_teste, preditos_teste)
print('accuracy: ', accuracy)
print('recall: ', recall)
print('precision: ', precision)
print('score: ', score)

[[20  0]
 [ 0 23]]
accuracy:  [1.0]
recall:  [1.0]
precision:  [1.0]
score:  [1.0]


In [13]:
# TODO: FLORESTAS ALEATÓRIAS

dados_originais = pd.read_csv('datasets/voting.csv')

n_trees = 15
my_tress = list()
my_cases_for_test = list()
for n in range(n_trees):
    n_bootstrap = bootstrap(dados_originais)       
    n_cross = cross_validation_split(n_bootstrap.values.tolist(), 10)
    my_cases_for_test.append(build_pre_dataset(test_split(n_cross, 10)))
    train_set = train_split(n_cross, 10)
    new_tree_from_train = build_tree(build_from_split(headers, new_train))
    my_tress.append(new_tree_from_train)

[INFO] Atributo:  handicapped-infants - entropia média:  0.8498982165269577
[INFO] Atributo:  handicapped-infants - ganho:  0.12837163833317433
--
[INFO] Atributo:  water-project-cost-sharing - entropia média:  0.9743548925747074
[INFO] Atributo:  water-project-cost-sharing - ganho:  0.00391496228542465
--
[INFO] Atributo:  adoption-of-the-budget-resolution - entropia média:  0.5741763181345978
[INFO] Atributo:  adoption-of-the-budget-resolution - ganho:  0.4040935367255343
--
[INFO] Atributo:  physician-fee-freeze - entropia média:  0.235918490457937
[INFO] Atributo:  physician-fee-freeze - ganho:  0.7423513644021951
--
[INFO] Atributo:  el-salvador-aid - entropia média:  0.5772960803759168
[INFO] Atributo:  el-salvador-aid - ganho:  0.40097377448421523
--
[INFO] Atributo:  religious-groups-in-schools - entropia média:  0.865645075995777
[INFO] Atributo:  religious-groups-in-schools - ganho:  0.11262477886435507
--
[INFO] Atributo:  anti-satellite-test-ban - entropia média:  0.8131542

[INFO] Atributo:  synfuels-corporation-cutback - ganho:  0.005341647265700952
--
[INFO] Atributo:  education-spending - entropia média:  0.04379200144324535
[INFO] Atributo:  education-spending - ganho:  0.0010576127081252731
--
[INFO] Atributo:  superfund-right-to-sue - entropia média:  0.04214992154275959
[INFO] Atributo:  superfund-right-to-sue - ganho:  0.002699692608611033
--
[INFO] Atributo:  crime - entropia média:  0.03602483839876575
[INFO] Atributo:  crime - ganho:  0.008824775752604867
--
[INFO] Atributo:  duty-free-exports - entropia média:  0.03724937767711321
[INFO] Atributo:  duty-free-exports - ganho:  0.0076002364742574074
--
[INFO] Atributo:  export-administration-act-south-africa - entropia média:  0.04183517323321366
[INFO] Atributo:  export-administration-act-south-africa - ganho:  0.0030144409181569604
--
----------------------------------------
[INFO] best_attribute:  adoption-of-the-budget-resolution
----------------------------------------
[INFO] Atributo:  han

[INFO] Atributo:  religious-groups-in-schools - entropia média:  0.1102358568612647
[INFO] Atributo:  religious-groups-in-schools - ganho:  0.0030384466358320122
--
[INFO] Atributo:  anti-satellite-test-ban - entropia média:  0.10506846137933307
[INFO] Atributo:  anti-satellite-test-ban - ganho:  0.008205842117763651
--
[INFO] Atributo:  aid-to-nicaraguan-contras - entropia média:  0.10925544131862086
[INFO] Atributo:  aid-to-nicaraguan-contras - ganho:  0.004018862178475857
--
[INFO] Atributo:  mx-missile - entropia média:  0.10748528573279043
[INFO] Atributo:  mx-missile - ganho:  0.005789017764306292
--
[INFO] Atributo:  immigration - entropia média:  0.09583740923569642
[INFO] Atributo:  immigration - ganho:  0.0174368942614003
--
[INFO] Atributo:  education-spending - entropia média:  0.06729931338733329
[INFO] Atributo:  education-spending - ganho:  0.04597499010976343
--
[INFO] Atributo:  superfund-right-to-sue - entropia média:  0.09372956156989017
[INFO] Atributo:  superfund-r

[INFO] Atributo:  export-administration-act-south-africa - ganho:  0.10817343634086418
--
----------------------------------------
[INFO] best_attribute:  adoption-of-the-budget-resolution
----------------------------------------
[INFO] Atributo:  handicapped-infants - entropia média:  0.19677767872596205
[INFO] Atributo:  handicapped-infants - ganho:  0.17445464791491358
--
[INFO] Atributo:  water-project-cost-sharing - entropia média:  0.3349968525637723
[INFO] Atributo:  water-project-cost-sharing - ganho:  0.036235474077103313
--
[INFO] Atributo:  el-salvador-aid - entropia média:  0.37123232664087563
[INFO] Atributo:  el-salvador-aid - ganho:  0.0
--
[INFO] Atributo:  religious-groups-in-schools - entropia média:  0.37123232664087563
[INFO] Atributo:  religious-groups-in-schools - ganho:  0.0
--
[INFO] Atributo:  anti-satellite-test-ban - entropia média:  0.3632975947985945
[INFO] Atributo:  anti-satellite-test-ban - ganho:  0.007934731842281106
--
[INFO] Atributo:  aid-to-nicarag

----------------------------------------
[INFO] best_attribute:  mx-missile
----------------------------------------
[INFO] Atributo:  handicapped-infants - entropia média:  0.36056805531517017
[INFO] Atributo:  handicapped-infants - ganho:  0.14269027946047558
--
[INFO] Atributo:  water-project-cost-sharing - entropia média:  0.5032583347756457
[INFO] Atributo:  water-project-cost-sharing - ganho:  0.0
--
[INFO] Atributo:  adoption-of-the-budget-resolution - entropia média:  0.2222222222222222
[INFO] Atributo:  adoption-of-the-budget-resolution - ganho:  0.28103611255342353
--
[INFO] Atributo:  el-salvador-aid - entropia média:  0.4601899388973658
[INFO] Atributo:  el-salvador-aid - ganho:  0.04306839587827993
--
[INFO] Atributo:  religious-groups-in-schools - entropia média:  0.3060986113514965
[INFO] Atributo:  religious-groups-in-schools - ganho:  0.19715972342414922
--
[INFO] Atributo:  anti-satellite-test-ban - entropia média:  0.4601899388973658
[INFO] Atributo:  anti-satellite-

...

[INFO] Atributo:  anti-satellite-test-ban - ganho:  0.0
--
[INFO] Atributo:  aid-to-nicaraguan-contras - entropia média:  0.9182958340544896
[INFO] Atributo:  aid-to-nicaraguan-contras - ganho:  0.0
--
[INFO] Atributo:  immigration - entropia média:  0.9182958340544896
[INFO] Atributo:  immigration - ganho:  0.0
--
[INFO] Atributo:  education-spending - entropia média:  0.9182958340544896
[INFO] Atributo:  education-spending - ganho:  0.0
--
[INFO] Atributo:  superfund-right-to-sue - entropia média:  0.9182958340544896
[INFO] Atributo:  superfund-right-to-sue - ganho:  0.0
--
[INFO] Atributo:  crime - entropia média:  0.9182958340544896
[INFO] Atributo:  crime - ganho:  0.0
--
[INFO] Atributo:  duty-free-exports - entropia média:  0.9182958340544896
[INFO] Atributo:  duty-free-exports - ganho:  0.0
--
[INFO] Atributo:  export-administration-act-south-africa - entropia média:  0.0
[INFO] Atributo:  export-administration-act-south-africa - ganho:  0.9182958340544896
--
------------------

[INFO] Atributo:  adoption-of-the-budget-resolution - ganho:  0.28103611255342353
--
[INFO] Atributo:  el-salvador-aid - entropia média:  0.4601899388973658
[INFO] Atributo:  el-salvador-aid - ganho:  0.04306839587827993
--
[INFO] Atributo:  religious-groups-in-schools - entropia média:  0.3060986113514965
[INFO] Atributo:  religious-groups-in-schools - ganho:  0.19715972342414922
--
[INFO] Atributo:  anti-satellite-test-ban - entropia média:  0.4601899388973658
[INFO] Atributo:  anti-satellite-test-ban - ganho:  0.04306839587827993
--
[INFO] Atributo:  aid-to-nicaraguan-contras - entropia média:  0.4831683939551968
[INFO] Atributo:  aid-to-nicaraguan-contras - ganho:  0.02008994082044896
--
[INFO] Atributo:  immigration - entropia média:  0.2222222222222222
[INFO] Atributo:  immigration - ganho:  0.28103611255342353
--
[INFO] Atributo:  education-spending - entropia média:  0.4601899388973658
[INFO] Atributo:  education-spending - ganho:  0.04306839587827993
--
[INFO] Atributo:  super

[INFO] Atributo:  synfuels-corporation-cutback - entropia média:  0.5509775004326938
[INFO] Atributo:  synfuels-corporation-cutback - ganho:  0.28566324150847355
--
[INFO] Atributo:  education-spending - entropia média:  0.0
[INFO] Atributo:  education-spending - ganho:  0.8366407419411673
--
[INFO] Atributo:  superfund-right-to-sue - entropia média:  0.331053480242907
[INFO] Atributo:  superfund-right-to-sue - ganho:  0.5055872616982604
--
[INFO] Atributo:  crime - entropia média:  0.5182958340544895
[INFO] Atributo:  crime - ganho:  0.3183449078866778
--
[INFO] Atributo:  duty-free-exports - entropia média:  0.7851454315650674
[INFO] Atributo:  duty-free-exports - ganho:  0.05149531037609989
--
[INFO] Atributo:  export-administration-act-south-africa - entropia média:  0.36731833362179583
[INFO] Atributo:  export-administration-act-south-africa - ganho:  0.4693224083193715
--
----------------------------------------
[INFO] best_attribute:  education-spending
-------------------------

[INFO] Atributo:  handicapped-infants - entropia média:  0.0
[INFO] Atributo:  handicapped-infants - ganho:  1.0
--
[INFO] Atributo:  el-salvador-aid - entropia média:  1.0
[INFO] Atributo:  el-salvador-aid - ganho:  0.0
--
[INFO] Atributo:  anti-satellite-test-ban - entropia média:  1.0
[INFO] Atributo:  anti-satellite-test-ban - ganho:  0.0
--
[INFO] Atributo:  aid-to-nicaraguan-contras - entropia média:  1.0
[INFO] Atributo:  aid-to-nicaraguan-contras - ganho:  0.0
--
[INFO] Atributo:  mx-missile - entropia média:  1.0
[INFO] Atributo:  mx-missile - ganho:  0.0
--
[INFO] Atributo:  immigration - entropia média:  1.0
[INFO] Atributo:  immigration - ganho:  0.0
--
[INFO] Atributo:  synfuels-corporation-cutback - entropia média:  1.0
[INFO] Atributo:  synfuels-corporation-cutback - ganho:  0.0
--
[INFO] Atributo:  education-spending - entropia média:  0.0
[INFO] Atributo:  education-spending - ganho:  1.0
--
[INFO] Atributo:  superfund-right-to-sue - entropia média:  0.0
[INFO] Atribut

[INFO] Atributo:  handicapped-infants - entropia média:  0.041160493779328615
[INFO] Atributo:  handicapped-infants - ganho:  0.0036891203720420054
--
[INFO] Atributo:  water-project-cost-sharing - entropia média:  0.038381350468119524
[INFO] Atributo:  water-project-cost-sharing - ganho:  0.0064682636832510965
--
[INFO] Atributo:  adoption-of-the-budget-resolution - entropia média:  0.02702890788795946
[INFO] Atributo:  adoption-of-the-budget-resolution - ganho:  0.01782070626341116
--
[INFO] Atributo:  el-salvador-aid - entropia média:  0.04310853043463173
[INFO] Atributo:  el-salvador-aid - ganho:  0.001741083716738888
--
[INFO] Atributo:  religious-groups-in-schools - entropia média:  0.039941010458922545
[INFO] Atributo:  religious-groups-in-schools - ganho:  0.0049086036924480755
--
[INFO] Atributo:  anti-satellite-test-ban - entropia média:  0.04328565848451442
[INFO] Atributo:  anti-satellite-test-ban - ganho:  0.001563955666856201
--
[INFO] Atributo:  aid-to-nicaraguan-contras

In [14]:
def forest_classify(trees, classes, input):
    votes = [classify(tree, classes, input) for tree in trees]
    vote_counts = Counter(votes)
    return vote_counts.most_common(1)[0][0]

classe_teste = list()
preditos_teste = list()
for current in build_from_split(headers, my_cases_for_test[0]):
    preditos_teste.append(current[1])
    classe_teste.append(forest_classify(my_tress, classes, current[0]))

KeyError: None

In [ ]:
classification_report(classe_teste, preditos_teste)
print('accuracy: ', accuracy)
print('recall: ', recall)
print('precision: ', precision)
print('score: ', score)

4. O mecanismo de amostragem de m atributos a cada divisão de nó, a partir dos quais será selecionado o melhor atributo de acordo com o critério de Ganho de Informação

5. O treinamento de um ensemble de árvores de decisão, adotando os mecanismos de bootstrap e seleção de atributos com amostragem, como mencionados acima


6. O mecanismo de votação majoritária entre as múltiplas árvores de decisão no ensemble, para classificação de novas instâncias utilizando o modelo de Florestas Aleatórias

7. A técnica de validação cruzada (cross-validation) estratificada, para avaliar poder de generalização do modelo e a variação de desempenho de acordo com diferentes valores para os parâmetros do algoritmo (ex., número de árvores no ensemble)

8. Avaliação do impacto do número de árvores no desempenho do ensemble